<a href="https://colab.research.google.com/github/Ayaansehgal/SignLanguageTranslator/blob/main/LAW_BOT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers datasets trl[sft] pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install transformers datasets trl accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from datasets import Dataset
import pandas as pd
import os

model_name = "distilgpt2"
dataset_csv = "indian_legal_faq_500.csv"
output_dir = "./distilgpt2-legalfaq-finetuned"

df = pd.read_csv(dataset_csv)

def format_example(example):
    return {"text": f"<|user|> {example['Question']} <|assistant|> {example['Answer']}"}

dataset = Dataset.from_pandas(df[["Question", "Answer"]])
dataset = dataset.map(format_example)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    fp16=False,
    no_cuda=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
)
print("Starting model training...")
trainer.train()
print("Training complete.")

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("Saved model files:")
print(os.listdir(output_dir))


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1577: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/500 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Starting model training...


Step,Training Loss
10,3.531300
20,2.042300
30,1.316100
40,1.161700
50,0.757400
60,0.620400
70,0.421700
80,0.291600
90,0.219100
100,0.208500


Training complete.
Saved model files:
['tokenizer_config.json', 'special_tokens_map.json', 'model.safetensors', 'runs', 'tokenizer.json', 'merges.txt', 'config.json', 'checkpoint-750', 'training_args.bin', 'vocab.json', 'generation_config.json']


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "./distilgpt2-legalfaq-finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

def ask_legal_question(question):
    prompt = f"<|user|> {question} <|assistant|>"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1].strip()

while True:
    user_input = input("Ask your legal question (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        print("Exiting. Thank you!")
        break
    answer = ask_legal_question(user_input)
    print(f"Answer: {answer}\n")


Ask your legal question (or type 'exit' to quit): whats the legal driving age 
Answer: An FIR is the first step to reporting a cognizable offense. You file it at the nearest police station by giving a written or verbal complaint.

Ask your legal question (or type 'exit' to quit): whats the legal driving age
Answer: driving under the influence is a Class A misdemeanor.

Ask your legal question (or type 'exit' to quit): what is fir
Answer: file online at the National Cyber Crime Reporting Portal or at a cybercrime police station.

Ask your legal question (or type 'exit' to quit): exit
Exiting. Thank you!
